In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [4]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="/home/zach/intelligent_machines/data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="/home/zach/intelligent_machines/data",
    train=False,
    download=True,
    transform=ToTensor(),
)

100.0%


Extracting /home/zach/intelligent_machines/data/FashionMNIST/raw/train-images-idx3-ubyte.gz to /home/zach/intelligent_machines/data/FashionMNIST/raw



100.0%


Extracting /home/zach/intelligent_machines/data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to /home/zach/intelligent_machines/data/FashionMNIST/raw



100.0%


Extracting /home/zach/intelligent_machines/data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to /home/zach/intelligent_machines/data/FashionMNIST/raw



100.0%

Extracting /home/zach/intelligent_machines/data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to /home/zach/intelligent_machines/data/FashionMNIST/raw



In [5]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")


Using cuda device


In [9]:

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [10]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [11]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [12]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [14]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 1.164709  [   64/60000]
loss: 1.167383  [ 6464/60000]
loss: 0.986616  [12864/60000]
loss: 1.121902  [19264/60000]
loss: 0.999743  [25664/60000]
loss: 1.029232  [32064/60000]
loss: 1.063544  [38464/60000]
loss: 1.001717  [44864/60000]
loss: 1.052400  [51264/60000]
loss: 0.984113  [57664/60000]
Test Error: 
 Accuracy: 65.0%, Avg loss: 0.991525 

Epoch 2
-------------------------------
loss: 1.049891  [   64/60000]
loss: 1.073355  [ 6464/60000]
loss: 0.877162  [12864/60000]
loss: 1.032091  [19264/60000]
loss: 0.910079  [25664/60000]
loss: 0.937562  [32064/60000]
loss: 0.985902  [38464/60000]
loss: 0.930661  [44864/60000]
loss: 0.974053  [51264/60000]
loss: 0.918228  [57664/60000]
Test Error: 
 Accuracy: 66.7%, Avg loss: 0.919626 

Epoch 3
-------------------------------
loss: 0.963040  [   64/60000]
loss: 1.006340  [ 6464/60000]
loss: 0.797402  [12864/60000]
loss: 0.967068  [19264/60000]
loss: 0.848259  [25664/60000]
loss: 0.868395  [32064/600

In [15]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [16]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth", weights_only=True))

<All keys matched successfully>

In [19]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
